In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import yaml
from collections import defaultdict
import pickle as pkl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import respy as rp
from respy.pre_processing.model_processing import process_params_and_options

from python.auxiliary_setup import *
from python.auxiliary_weighting import *
from python.auxiliary_plots import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simulated Method of Moments estimation

In this notebook we explore the estimation of model parameters in a simple discrete choice dynamic programming model using the simulated method of moments (SMM). The notebook includes the basic estimation procedure as well as an estimation exercise using different specifications.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Observed-Data" data-toc-modified-id="Observed-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Observed Data</a></span></li><li><span><a href="#Data-Moments" data-toc-modified-id="Data-Moments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Moments</a></span></li><li><span><a href="#Weighting-Matrix" data-toc-modified-id="Weighting-Matrix-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Weighting-Matrix</a></span></li><li><span><a href="#Criterion-Function" data-toc-modified-id="Criterion-Function-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Criterion Function</a></span></li><li><span><a href="#Estimation-Exercise" data-toc-modified-id="Estimation-Exercise-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Estimation Exercise</a></span><ul class="toc-item"><li><span><a href="#Estimation-for-one-parameter" data-toc-modified-id="Estimation-for-one-parameter-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Estimation for one parameter</a></span><ul class="toc-item"><li><span><a href="#Simulated-Moments" data-toc-modified-id="Simulated-Moments-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Simulated Moments</a></span></li><li><span><a href="#Optimization-procedure" data-toc-modified-id="Optimization-procedure-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Optimization procedure</a></span></li></ul></li><li><span><a href="#Estimation-of-multiple-parameters" data-toc-modified-id="Estimation-of-multiple-parameters-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Estimation of multiple parameters</a></span></li><li><span><a href="#Changing-the-Simulation-Seed" data-toc-modified-id="Changing-the-Simulation-Seed-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Changing the Simulation Seed</a></span></li><li><span><a href="#Fixing-one-parameter-at-the-wrong-value" data-toc-modified-id="Fixing-one-parameter-at-the-wrong-value-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fixing one parameter at the wrong value</a></span></li><li><span><a href="#Retrieving-the-true-parameter-vector" data-toc-modified-id="Retrieving-the-true-parameter-vector-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Retrieving the true parameter vector</a></span></li></ul></li></ul></div>

## Introduction

The simulated method of moments approach to estimating model parameters is to minimize a certain distance between observed moments and simulated moments with respect to the parameters that generate the simulated model.

Denote $X$ our observed data and $m(X)$ the vector of observed moments. To construct the criterion function, we use the parameter vector $\theta$ to simulate data from the  model $\hat{X}$. We can then calculate the simulated moments $m(\hat{X}| \theta)$.

The criterion function is then given by 

\begin{equation}
\psi(\theta) = (m(X) - m(\hat{X}| \theta))'\Omega(m(X) - m(\hat{X}| \theta))
\end{equation}

where the difference between observed and simulated moments $(m(X) - m(\hat{X}| \theta))$ constitutes a vector of the dimension $1\times M$ with $1,..,M$ denoting the number of moments. The $M\times M$ weighting matrix is given by $\Omega$. 

The SMM estimator is defined as the solution to 

\begin{equation}
\hat{\theta}(\Omega) = \underset{\theta}{\operatorname{argmin}} \psi(\theta).
\end{equation}

The criterion function is thus a strictly positive scalar and the estimator depends on the choice of moments $m$ and the weighting matrix $\Omega$. The weighting matrix applies some scaling for discrepancies between the observed and simulated moments. If we use the identity matrix, each moment is given equal weight and the criterion function reduces to the sum of squared moment deviations. 

Aside from the choice of moments and weighting matrix, some other important choices that influence the the estimation are the simulator itself and the algorithm and specifications for the optimization procedure. Many explanations for simulated method of moments estimation also feature the number of simulations as a factor that is to be determined for estimation. We can ignore this factor for now since we are working with a large simulated dataset.

In the following we will set up the data, moments and weighting matrix needed to construct the criterion function and subsequently try to estimating the parameters of the model using this SMM setup.

##  Observed Data

We generate our model and data using [respy](https://respy.readthedocs.io/en/latest/) and a simple Robinson Crusoe model. In this model, the agent, Robinson Crusoe, in each time period decides between two choice options: working (i.e. going fishing) or spending time in the hammock. 

We can use respy to simulate the data for this exercise.

Let's take a look at the model specifications.

## Data Moments

For the setup of the estimation we first have to choose a set of moments that we will use to match the observed data and the simulated model. For this model we include two sets of moments: 

1. The first set are Robinson's **choice frequencies** (choice frequencies here refers to the share of agents that have chosen a specific option) for each period. 
2. The second set are moments that characterize the **wage distribution** for each period, i.e. the mean of the wage of all agents that have chosen fishing in a given period and the standard deviation of the wages. 

In addition to the data, we need the complete set of potential choice options for Robinson. Respy lets us extract them from the model parameters and options.

We need a function that computes the set of moments on the observed and simulated data.

Now we are ready to calculate the moments.

## Weighting Matrix

Next we specify a weighting matrix. It needs to be a square matrix with the same number of diagonal elements as there are moments. One option would be to use the identity matrix, but we use a weighting matrix that adjusts for the variance of each moment. The variances for the moments are constructed using a bootstrapping procedure. 

## Criterion Function 

We have collected the observed data for our model, chosen the set of moments we want to use for estimation and defined a weighting matrix based on these moments. We can now set up the criterion function to use for estimation. 

As already discussed above, the criterion function is given by the weighted square product of the difference between observed moments $m(X)$ and simulated moments $m(\hat{X}| \theta)$. Trivially, if we have that $m(X) = m(\hat{X}| \theta)$, the criterion function returns a value of 0. Thus, the closer $\theta$ is to the real parameter vector, the smaller should be the value for the criterion function. 

Criterion function at the true parameter vector:

We can plot the criterion function to examine its behavior around the minimum in more detail. The plots below show the criterion function at varying values of all parameters in the the paramter vector.

This depiction conceals the fact that the criterion function is not a smooth function of our parameter values. We can reveal this property if we 'zoom into' the function far enough. The plots below show the criterion function for varying values of *delta* around the true minimum value of 0.95. We can see that the function exhibits small plateaus and is thus not completely smooth. 

##  Estimation Exercise

In the following we will conduct a simulation exercise to estimate the parameter vector using our criterion function and weighting matrix. We will begin by simulating data using the new parameter vector and examine how the simulated moments differ from the observed ones. We will then use an optimizer to minimize the criterion function in order to retrieve the true parameter vector. Additionally, we will explore how the criterion function behaves if we change the simulation seed or misspecify the constraints by fixing parameters at the wrong values.

### Estimation for one parameter

For now, our candidate parameter vector will just differ in *delta* from the true parameters.

#### Simulated Moments

We can now use our model to simulate data using the candidate parameter vector. We can see that the choice frequencies and wage distribution differ from the moments of the observed dataset.

We can plot the moments to compare the choice frequencies for each period.

The plots below show the mean and the standard deviation in the wages for each period.

The criterion function value for the candidate parameter vector is not zero.

#### Optimization procedure

We will now use an optimization procedure to retrieve the true parameter vector. For the optimization we can use [estimagic](https://estimagic.readthedocs.io/en/latest/index.html). In order to minimize the criterion function we need estimagic's `minimize` function.



In [ ]:
from estimagic.optimization.optimize import minimize

We have verified above that the criterion function gives a value of 0 for the true parameter vector. Before we try different parameter specifications, we can check whether an optimizer recognizes the true vector as the minimum of our criterion function.

As the code below shows, the optimization algorithm recognizes the true parameter vector as the minimum of the criterion function as it returns a function value of 0 and the true parameter values.

##### Upper and Lower Bounds

We can help the optimizer by specifying bounds for the parameters. Since we know the true parameters in the case of this model, we can just pick upper and lower bounds that are fairly close to the true values of the parameters to aid the optimizer in the search for the optimum. By default, the upper and lower bounds are set to $\infty$ and $-\infty$, so specifying upper and lower bounds substantially reduces the range of parameter values that the optimizer can potentially cover.

For optimization with estimagic, we can specify bounds by adding the columns *'lower'* and *'upper'* to the dataframe that contains the parameter values.

In [138]:
params_cand['lower'] = [0.89, 0.066, -0.11, 1.04, 0, 0, 0]
params_cand['upper'] = [0.98, 0.072, -0.095, 1.055, 0.1, 0.1, 0.1]
params_cand

value  lower  upper
category       name                                     
delta          delta                 0.930  0.890  0.980
wage_fishing   exp_fishing           0.070  0.066  0.072
nonpec_fishing constant             -0.100 -0.110 -0.095
nonpec_hammock constant              1.046  1.040  1.055
shocks_sdcorr  sd_fishing            0.010  0.000  0.100
               sd_hammock            0.010  0.000  0.100
               corr_hammock_fishing  0.000  0.000  0.100

##### Constraints

Additionally we hold all other parameters fixed for now to aid the optimizer in finding the optimal value for *delta*.

In [139]:
# Define base constraints to use for the rest of the notebook.
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"}, 
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type":"fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

##### Optimize

We can now optimize the criterion function with respect to the parameter vector. The optimizer manages to reach a function value of 0 and finds an approximately correct *delta* for our model. 

This exercise again reveals that we are dealing with a non-smooth criterion function. The optimizer does not return the exact value of 0.95 for *delta* because of the little plateaus we saw when zooming into the criterion function. As the plot shows, there is a small area around the true value for *delta* that also returns a function value of 0 and might thus be picked by the optimizer.

### Changing the Simulation Seed

As shown above, the optimizer manages to find a function value of exactly 0 for the true parameter vector. This is the case because respy controls the realization of random elements in the simulation process via a simulation seed. The model thus always simulates the exact same dataset for a given parameter vector. Our criterion function becomes exactly 0 at the true parameter vector because for this vector, the observed and simulated data are identical.

Changing the simulation seed results in simulated moments that, even for the true parameter vector, are never completely equal to the observed moments. 

Let's estimate the model with a different simulation seed.

We can see that the criterion function is not exactly 0 anymore for the true parameter vector.

Our optimizer thus also does not return a function value of 0 for the true parameter vector.

Since the optimizer does not even recognize the true parameter vector, it is also not able to reach a criterion function  value of 0 for a different parameter vector.

### Fixing one parameter at the wrong value

We change the parameters value for *delta* and *wage_fishing* and fix the latter in the constraints.

In [147]:
params_cand = params_true.copy()
params_cand.loc["delta", "value"] = 0.93
params_cand.loc["wage_fishing", "value"] = 0.072

params_cand['lower'] = [0.89, 0.066, -0.11, 1.04, 0, 0, 0]
params_cand['upper'] = [0.98, 0.072, -0.095, 1.055, 0.1, 0.1, 0.1]

The optimizer cannot retrieve the true parameter and does not reach a value of 0.

### Retrieving the true parameter vector

We have seen above that misspecifying the constraints by fixing a parameter at a value that is not optimal prevents the criterion function in reaching a value of 0 when estimating other parameters. We now repeat the estimation with the new parameter vector.

The parameter for *wage_fishing* is still 0.072 since we fixed it for the prior estimation:

We now free up *wage_fishing* in the constraints in addition to *delta*.

In [150]:
# Adjust constraints to free up both delta and wage_fishing.
constr_u = constr_base.copy()
constr_u.remove({'loc': 'delta', 'type': 'fixed'})
constr_u.remove({'loc': 'wage_fishing', 'type': 'fixed'})

Freeing up the non-optimal *wage_fishing* improves the estimates. The criterion function value is much closer to 0 and the optimizer manages to retrieve the true parameter values quite closely.

For easier comparison, we can compute the difference between the true and estimated value:

## References

* Adda, J., & Cooper, R. W. (2003). *Dynamic economics: quantitative methods and applications*. MIT press.


* Davidson, R., & MacKinnon, J. G. (2004). *Econometric theory and methods (Vol. 5)*. New York: Oxford University Press.


* Evans, R. W. (2018, July 5). Simulated Method of Moments (SMM) Estimation. Retrieved November 30, 2019, from https://notes.quantecon.org/submission/5b3db2ceb9eab00015b89f93.


* Gourieroux, M., & Monfort, D. A. (1996). *Simulation-based econometric methods*. Oxford university press.


* McFadden, D. (1989). A method of simulated moments for estimation of discrete response models without numerical integration. *Econometrica: Journal of the Econometric Society*, 995-1026.